طبق قوانین بازی، برای برنده شدن بهتر است انتخاب ها بیشتر روی ستون وسط باشد، بنابراین به piece های ستون وسط امتیاز بیشتری تعلق می‌گیرد. قانون دیگر بازی این است که باید ردیف های چهارتایی توسط یک بازیگر پر شود. از این قانون نیز برای محاسبه امتیاز استفاده شده است. مثلا اگر در یک پنجره چهارتایی تعداد piece های هریک از بازیکنان دو و دو خانه ی دیگر پنجره خالی باشد امتیاز دو برای بازیگر در نظر گرفته شده است زیرا این بازیگر فرصت دارد در حرکت های بعدی برنده شود ولی اگر سه خانه پنجره توسط حریف پر شده باشد بازیگر با امتیاز 4- جریمه خواهد شد.

In [16]:
def evaluate_window(self, window, piece):
    score = 0
    opp_piece = self.CPU
    if piece == self.CPU:
        opp_piece = self.YOU

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(self.EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(self.EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(self.EMPTY) == 1:
        score -= 4

    return score

In [ ]:
def heuristic(self, piece):
    score = 0

    # Score center column
    center_array = [int(row[self.columns // 2]) for row in self.board]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(self.rows):
        row_array = self.board[r]
        for c in range(self.columns - 3):
            window = row_array[c:c + self.__CONNECT_NUMBER]
            score += self.evaluate_window(window, piece)

    # Score Vertical
    for c in range(self.columns):
        col_array = [int(row[c]) for row in self.board]
        for r in range(self.rows - 3):
            window = col_array[r:r + self.__CONNECT_NUMBER]
            score += self.evaluate_window(window, piece)

    # Score posiive sloped diagonal
    for r in range(self.rows - 3):
        for c in range(self.columns - 3):
            window = [self.board[r + i][c + i] for i in range(self.__CONNECT_NUMBER)]
            score += self.evaluate_window(window, piece)

    for r in range(self.rows - 3):
        for c in range(self.columns - 3):
            window = [self.board[r + 3 - i][c + i] for i in range(self.__CONNECT_NUMBER)]
            score += self.evaluate_window(window, piece)

    return score

In [1]:
def minimax(self, depth, maximizingPlayer):
    valid_locations = self.get_possible_moves()
    is_terminal = self.is_terminal_node()
    if depth == 0 or is_terminal:
        if is_terminal:
            if self.check_if_player_has_won(self.YOU):
                return (None, 100000000000000)
            elif self.check_if_player_has_won(self.CPU):
                return (None, -10000000000000)
            else:
                return (None, 0)
        else:  # Depth is zero
            return (None, self.heuristic(self.YOU))
    if maximizingPlayer:
        value = -math.inf
        column = choice(valid_locations)
        for col in valid_locations:
            #row = self.get_next_open_row(col)
            b_copy = copy.deepcopy(self.board)
            self.__drop_piece_in_column(col, self.YOU)
            new_score = self.minimax(depth - 1, False)[1]
            self.board = copy.deepcopy(b_copy)
            if new_score > value:
                value = new_score
                column = col
        return column, value

    else:  # Minimizing player
        value = math.inf
        column = choice(valid_locations)
        for col in valid_locations:
            b_copy = copy.deepcopy(self.board)
            self.__drop_piece_in_column(col, self.CPU)
            new_score = self.minimax(depth - 1, True)[1]
            self.board = copy.deepcopy(b_copy)
            if new_score < value:
                value = new_score
                column = col
        return column, value

In [2]:
def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        # TODO: complete here
        b_copy = copy.deepcopy(self.board)
        col, _  = self.minimax(5, True)
        self.board = copy.deepcopy(b_copy)
        return  col


In [60]:
from random import random, choice
import copy
import math
import time

class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"

        self.__init_board()
        turns_played = 0
        current_player = starter
        while (turns_played < self.board_size):

            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        # TODO: complete here
        b_copy = copy.deepcopy(self.board)
        col, _  = self.minimax(1, True)
        self.board = copy.deepcopy(b_copy)
        return  col

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
                self.__has_player_won_diagonally(player_id)
                or self.__has_player_won_horizentally(player_id)
                or self.__has_player_won_vertically(player_id)
        )

    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0

    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves

    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]

    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]

    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True

    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")

    def __change_turn(self, turn):
        if (turn == self.YOU):
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"

                print(f"{house_char}", end=" ")
            print()

    def is_terminal_node(self):
        return self.check_if_player_has_won(self.CPU) or self.check_if_player_has_won(self.YOU) or \
               len(self.get_possible_moves()) == 0

    def get_next_open_row(self, col):
        for r in range(self.rows):
            if self.board[r][col] == 0:
                return r

    def evaluate_window(self, window, piece):
        score = 0
        opp_piece = self.CPU
        if piece == self.CPU:
            opp_piece = self.YOU

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(self.EMPTY) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(self.EMPTY) == 2:
            score += 2

        if window.count(opp_piece) == 3 and window.count(self.EMPTY) == 1:
            score -= 4

        return score

    def heuristic(self, piece):
        score = 0

        #Score center column
        center_array = [int(row[self.columns // 2]) for row in self.board]
        center_count = center_array.count(piece)
        score += center_count * 3

        #Score Horizontal
        for r in range(self.rows):
            row_array = self.board[r]
            for c in range(self.columns - 3):
                window = row_array[c:c + self.__CONNECT_NUMBER]
                score += self.evaluate_window(window, piece)

        #Score Vertical
        for c in range(self.columns):
            col_array = [int(row[c]) for row in self.board]
            for r in range(self.rows - 3):
                window = col_array[r:r + self.__CONNECT_NUMBER]
                score += self.evaluate_window(window, piece)

        #Score posiive sloped diagonal
        for r in range(self.rows - 3):
            for c in range(self.columns - 3):
                window = [self.board[r + i][c + i] for i in range(self.__CONNECT_NUMBER)]
                score += self.evaluate_window(window, piece)

        for r in range(self.rows - 3):
            for c in range(self.columns - 3):
                window = [self.board[r + 3 - i][c + i] for i in range(self.__CONNECT_NUMBER)]
                score += self.evaluate_window(window, piece)

        return score

    def minimax(self, depth, maximizingPlayer):
        valid_locations = self.get_possible_moves()
        is_terminal = self.is_terminal_node()
        if depth == 0 or is_terminal:
            if is_terminal:
                if self.check_if_player_has_won(self.YOU):
                    return (None, 100000000000000)
                elif self.check_if_player_has_won(self.CPU):
                    return (None, -10000000000000)
                else:
                    return (None, 0)
            else:
                return (None, self.heuristic(self.YOU))
        if maximizingPlayer:
            value = -math.inf
            column = choice(valid_locations)
            for col in valid_locations:
                #row = self.get_next_open_row(col)
                b_copy = copy.deepcopy(self.board)
                self.__drop_piece_in_column(col, self.YOU)
                new_score = self.minimax(depth - 1, False)[1]
                self.board = copy.deepcopy(b_copy)
                if new_score > value:
                    value = new_score
                    column = col
            return column, value

        else:  # Minimizing player
            value = math.inf
            column = choice(valid_locations)
            for col in valid_locations:
                b_copy = copy.deepcopy(self.board)
                self.__drop_piece_in_column(col, self.CPU)
                new_score = self.minimax(depth - 1, True)[1]
                self.board = copy.deepcopy(b_copy)
                if new_score < value:
                    value = new_score
                    column = col
            return column, value


# if __name__ == '__main__':
#     board_sizes_to_check = [(6, 7),
#                             (7, 8),
#                             (7, 10)]
#     game = ConnectSin(board_size=(7, 10), silent=False)
#     game.run()

In [61]:
if __name__ == '__main__':
    board_sizes_to_check = [(6, 7),
                            (7, 8),
                            (7, 10)]
    game = ConnectSin(board_size=(6, 7), silent=False)

    start = time.time()
    game.run()
    end = time.time()

    print(f'Time: {(end - start):.2f} seconds')

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O C Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O C Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O O C Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y Y O O 
C O C Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O Y C O O O 
O O Y Y Y O O 
C C C Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O Y C O O O 
O Y Y Y Y O O 
C C C Y C O O 
congrats! you have won!
Time: 0.05 seconds


## minimax
نتیجه‌گیری کلی در جدول زیر آمده است

|            |Depth = 1|Depth = 3|Depth = 5|
|:-----------|:-------:|:-------:|:-------:|
|6x7         |0.03s    |0.97s    |48.97s   |
|7x8         |0.04s    |1.97s    |169.10s  |
|7x10        |0.09s    |10.82s   |428.57s  |

## هرس آلفا و بتا
در این قسمت تابع minimax مانند کد  زیر تغییر جزئی خواهد کرد و از آرگومان های آلفا و بتا برای هرس کردن الگوریتم minimax استفاده شده است.

col, _  = self.minimax(4, -math.inf, math.inf, True)

تکه کد بالا برای فراخوانی متاد minimax در تابع get_your_input استفاده شده است. آلفا منهای بی‌نهایت و بتا مثبت بی‌نهایت در نظر گرفته شده است.

In [ ]:
def minimax(self, depth, alpha, beta, maximizingPlayer):
    valid_locations = self.get_possible_moves()
    is_terminal = self.is_terminal_node()
    if depth == 0 or is_terminal:
        if is_terminal:
            if self.check_if_player_has_won(self.YOU):
                return (None, 100000000000000)
            elif self.check_if_player_has_won(self.CPU):
                return (None, -10000000000000)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, self.heuristic(self.YOU))
    if maximizingPlayer:
        value = -math.inf
        column = choice(valid_locations)
        for col in valid_locations:
            #row = self.get_next_open_row(col)
            b_copy = copy.deepcopy(self.board)
            self.__drop_piece_in_column(col, self.YOU)
            new_score = self.minimax(depth - 1, alpha, beta, False)[1]
            self.board = copy.deepcopy(b_copy)
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else:  # Minimizing player
        value = math.inf
        column = choice(valid_locations)
        for col in valid_locations:
            b_copy = copy.deepcopy(self.board)
            self.__drop_piece_in_column(col, self.CPU)
            new_score = self.minimax(depth - 1, alpha, beta, True)[1]
            self.board = copy.deepcopy(b_copy)
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

In [ ]:
from random import random, choice
import copy
import math
import time

class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"

        self.__init_board()
        turns_played = 0
        current_player = starter
        while (turns_played < self.board_size):

            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        # TODO: complete here
        b_copy = copy.deepcopy(self.board)
        col, _  = self.minimax(3, -math.inf, math.inf, True)
        self.board = copy.deepcopy(b_copy)
        return  col

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
                self.__has_player_won_diagonally(player_id)
                or self.__has_player_won_horizentally(player_id)
                or self.__has_player_won_vertically(player_id)
        )

    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0

    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves

    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]

    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]

    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True

    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")

    def __change_turn(self, turn):
        if (turn == self.YOU):
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"

                print(f"{house_char}", end=" ")
            print()

    def is_terminal_node(self):
        return self.check_if_player_has_won(self.CPU) or self.check_if_player_has_won(self.YOU) or \
               len(self.get_possible_moves()) == 0

    def get_next_open_row(self, col):
        for r in range(self.rows):
            if self.board[r][col] == 0:
                return r

    def evaluate_window(self, window, piece):
        score = 0
        opp_piece = self.CPU
        if piece == self.CPU:
            opp_piece = self.YOU

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(self.EMPTY) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(self.EMPTY) == 2:
            score += 2

        if window.count(opp_piece) == 3 and window.count(self.EMPTY) == 1:
            score -= 4

        return score

    def heuristic(self, piece):
        score = 0

        ## Score center column
        center_array = [int(row[self.columns // 2]) for row in self.board]
        center_count = center_array.count(piece)
        score += center_count * 3

        #Score Horizontal
        for r in range(self.rows):
            row_array = self.board[r]
            for c in range(self.columns - 3):
                window = row_array[c:c + self.__CONNECT_NUMBER]
                score += self.evaluate_window(window, piece)

        #Score Vertical
        for c in range(self.columns):
            col_array = [int(row[c]) for row in self.board]
            for r in range(self.rows - 3):
                window = col_array[r:r + self.__CONNECT_NUMBER]
                score += self.evaluate_window(window, piece)

        #Score posiive sloped diagonal
        for r in range(self.rows - 3):
            for c in range(self.columns - 3):
                window = [self.board[r + i][c + i] for i in range(self.__CONNECT_NUMBER)]
                score += self.evaluate_window(window, piece)

        for r in range(self.rows - 3):
            for c in range(self.columns - 3):
                window = [self.board[r + 3 - i][c + i] for i in range(self.__CONNECT_NUMBER)]
                score += self.evaluate_window(window, piece)

        return score

    def minimax(self, depth, alpha, beta, maximizingPlayer):
        valid_locations = self.get_possible_moves()
        is_terminal = self.is_terminal_node()
        if depth == 0 or is_terminal:
            if is_terminal:
                if self.check_if_player_has_won(self.YOU):
                    return (None, 100000000000000)
                elif self.check_if_player_has_won(self.CPU):
                    return (None, -10000000000000)
                else:  # Game is over, no more valid moves
                    return (None, 0)
            else:  # Depth is zero
                return (None, self.heuristic(self.YOU))
        if maximizingPlayer:
            value = -math.inf
            column = choice(valid_locations)
            for col in valid_locations:
                #row = self.get_next_open_row(col)
                b_copy = copy.deepcopy(self.board)
                self.__drop_piece_in_column(col, self.YOU)
                new_score = self.minimax(depth - 1, alpha, beta, False)[1]
                self.board = copy.deepcopy(b_copy)
                if new_score > value:
                    value = new_score
                    column = col
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return column, value

        else:  # Minimizing player
            value = math.inf
            column = choice(valid_locations)
            for col in valid_locations:
                b_copy = copy.deepcopy(self.board)
                self.__drop_piece_in_column(col, self.CPU)
                new_score = self.minimax(depth - 1, alpha, beta, True)[1]
                self.board = copy.deepcopy(b_copy)
                if new_score < value:
                    value = new_score
                    column = col
                beta = min(beta, value)
                if alpha >= beta:
                    break
            return column, value
        
# if __name__ == '__main__':
#     board_sizes_to_check = [(6, 7),
#                             (7, 8),
#                             (7, 10)]
#     game = ConnectSin(board_size=(7, 10), silent=False)
#     game.run()

In [63]:
if __name__ == '__main__':
    board_sizes_to_check = [(6, 7),
                            (7, 8),
                            (7, 10)]
    game = ConnectSin(board_size=(6, 7), silent=False)

    start = time.time()
    game.run()
    end = time.time()

    print(f'Time: {(end - start):.2f} seconds')

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O Y Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O Y C O O O 
O O Y Y C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O Y C O O O 
O O Y C O O O 
O O Y Y C O C 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O Y O O O O 
O O Y C O O O 
O O Y C O O O 
O O Y Y C O C 
congrats! you have won!
Time: 0.04 seconds


## minimax با هرس آلفا و بتا
نتیجه‌گیری کلی در جدول زیر آمده است

|            |Depth = 1|Depth = 3|Depth = 5|Depth = 7|
|:-----------|:-------:|:-------:|:-------:|:-------:|
|6x7         |0.04s    |0.49s    |3.19s    |71.04s   |
|7x8         |0.07s    |0.78s    |17.24s   |450.03s  |
|7x10        |0.14s    |3.41s    |37.24s   |1528.53s |

In [66]:
if __name__ == '__main__':
    board_sizes_to_check = [(6, 7),
                            (7, 8),
                            (7, 10)]
    game = ConnectSin(board_size=(6, 7), silent=False)
    tmp = []
    for i in range(200):
        print(f'{i}th race ...')
        tmp.append(game.run())
    win_rate = sum(tmp) / len(tmp)
    print(f'My win rate is: {win_rate}')


0th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O C O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O C Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O Y O O O 
O O O C O O O 
O C C Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O Y O O O 
O O O C O O O 
O C C Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O Y O O O 
O O Y C O O O 
O C C Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O Y Y O O O 
O C Y C O O O 
O C C Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O O Y Y O O O 
O C Y C O O O 
O C C Y O C O 
Y: you, C: CPU
O O O C O O O 
O O Y C O O O 
O O Y Y O O O 
O O Y Y O O O 
O C Y C O O O 
O C C Y O C O 
congrats! you have won!
1th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: yo

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O Y Y O C O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y Y Y O C O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y Y Y Y O C O 
congrats! you have won!
10th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O Y C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
C O Y C O O O 
Y: you, C: CPU
O 

Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O Y C O O O 
O O Y Y O O O 
C O Y C C O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O Y C O O O 
C O Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O C Y C O O O 
C Y Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O C Y C O O O 
C Y Y Y O O O 
C Y Y C C O O 
congrats! you have won!
19th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O Y Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O C C O O O 
O Y Y Y O O O 
Y: you, C: CPU
O 

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y Y Y Y O O O 
congrats! you have won!
30th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O C Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O C Y Y O O O 
Y: you, C: CPU
O 

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O Y C O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O O Y C O O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
C O Y C O O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
O O Y C O O O 
O O Y Y O O O 
C O Y C C O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O C C O O O 
O O Y C O O O 
O O Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O C C O O O 
O O Y C O O O 
O Y Y Y O O O 
C Y Y C C O C 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O C C O O O 
O O Y C O O O 
Y Y Y Y O O O 
C Y Y C C O C 
congrats! you have won!
41th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O 

...

Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
C O Y C O O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
O O Y C O O O 
O O Y Y O O O 
C O Y C C O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
O O Y C O O O 
C O Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
O O O Y O O O 
O O O C O O O 
O O O C O O O 
C O Y C O O O 
C Y Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
O O O Y O O O 
C O O C O O O 
Y O O C O O O 
C O Y C O O O 
C Y Y Y O O O 
C Y Y C C O O 
Y: you, C: CPU
Y O O Y O O O 
C O O C O O O 
Y O O C O O O 
C O Y C O O O 
C Y Y Y O O O 
C Y Y C C C O 
Y: you, C: CPU
Y O O Y O O O 
C O O C O O O 
Y O Y C O O O 
C O Y C O O O 
C Y Y Y O O O 
C Y Y C C C O 
congrats! you have won!
165th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y: you, C: CPU
O

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O C Y C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O Y Y Y O O O 
O C Y C C O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O C O O O 
O Y Y Y O O O 
Y C Y C C O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O C O O O 
Y Y Y Y O O O 
Y C Y C C O O 
congrats! you have won!
173th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y Y Y O O O 
Y: you, C: CPU
O

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y O Y O O C 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y Y Y O O C 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y Y Y Y O O C 
congrats! you have won!
181th race ...
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O Y O O C 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O Y Y C O C 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O Y Y C O C 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O O 
O O Y Y C O C 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O Y Y O O C 
Y O Y Y C O C 
Y: you, C: CPU
O

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O Y O O O 
O O O C O O O 
O C O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O Y O C O O O 
O C O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O Y O Y O O O 
O Y O C O O O 
O C O Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O Y O Y O O O 
O Y O C O O O 
O C O Y O Y O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O Y O C O O O 
O Y O Y O O O 
O Y O C O O O 
C C O Y O Y O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O Y O C O O O 
C Y O Y O O O 
Y Y O C O O O 
C C O Y O Y O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
Y Y O C O O O 
C Y O Y O O O 
Y Y O C O O O 
C C C Y O Y O 
Y: you, C: CPU
O O O C O O O 
Y O O C O O O 
Y Y O C O O O 
C Y O Y O O O 
Y Y O C O C O 
C C C Y O Y O 
Y: you, C: CPU
Y O O C O O O 
Y O O C O O O 
Y Y O C O 

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O O Y O O O 
Y: you, C: CPU
O O O O O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O O Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
O Y Y Y O O O 
Y: you, C: CPU
O O O C O O O 
O O O C O O O 
O O O C O O O 
O O O Y O O O 
O O O C O O O 
Y Y Y Y O O O 
congrats! you have won!
My win rate is: 0.99


برنامه با عمق 3 و صفحه بازی 6x7 پس از اجرای 200 مرتبه به win_rate برابر با 0.99 رسیده است.

# سوال 1
در بسیاری از مسائل، می‌توان هزینه رسیدن از یک وضعیت به وضعیت هدف را تخمین زد اما نمی‌توان آن را بطور دقیق تعیین نمود. پس اگر برای حل مسئله بتوان براساس بررسی تخمین‌هایی که بدست آورده شده به سمت مسیر درست حرکت کرد شانس توسعه نود‌های اشتباه بطور محسوسی کاهش می‌یابد. بطور خلاصه تابع heuristic باید تا حد امکان تخمینی از کم هزینه‌ترین مسیر از وضعیت جاری به وضعیت هدف باشد. در این مسئله تابع heuristic مشخص کننده‌ی وضعیت صفحه‌ی بازی خواهد بود که مقدار آن با هر حرکت بازیکنان تغییر می‌کند. انتخاب ستون وسط توسط بازیکن یک امتیاز ویژه می‌باشد که ترجیح الگوریتم انتخاب این ستون است.


# سوال 2
با افزایش عمق الگوریتم زمان اجرا و دقت آن افزایش می‌یابد. با توجه به جدول‌هایی که بررسی شد نتیجه گرفته می‌شود عمق الگوریتم تاثیر مستقیمی بر روی افزایش شانس پیروزی، زمان و گره‌های دیده شده می‌گذارد. 


# سوال 3
وقتی از روش هرس کردن استفاده می‌شود برای هر گره‌ی درخت اگر در ردیف maximizing باشد عدد minimax با آلفای والد آن مقایسه می‌شود و اگر این مقدار کمتر از آلفای والد باشد پیشروی از این گره انجام نمی‌شود و به عبارتی این شاخه از همین نقطه هرس خواهد شد. بطور مشابه اگر گره‌ی درخت در ردیف minimizing باشد عدد minimax با بتای والد خود مقایسه می‌شود و اگر مقداری بزرگتر از آن داشته باشد شاخه از همان نقطه هرس خواهد شد.